**HIER DER TEIL MIT DER KARTE DER SCHWEIZ**

Karten mit Kantonsname beim hoovering

In [100]:
import plotly.express as px
import geopandas as gpd
import pandas as pd

# Read your GeoDataFrame
geofilePATH = 'https://raw.githubusercontent.com/sawubona-repo/BINA-FS24-WORK/master/zDiversExamples/Notebook-GeoMapping/DATA/'
geofileNAME = 'ch-cantons-new.geojson'
geodf = gpd.read_file(geofilePATH + geofileNAME)

# Define the canton mapping
kanton_mapping = {
    1: 'ZH', 2: 'BE', 3: 'LU', 4: 'UR', 5: 'SZ', 6: 'OW', 7: 'NW', 8: 'GL', 9: 'ZG', 10: 'FR', 11: 'SO', 12: 'BS',
    13: 'BL', 14: 'SH', 15: 'AR', 16: 'AI', 17: 'SG', 18: 'GR', 19: 'AG', 20: 'TG', 21: 'TI', 22: 'VD', 23: 'VS',
    24: 'NE', 25: 'GE', 26: 'JU'
}

# Initialize a dictionary to hold figures for each year
figures = {}

# Loop through each year from 2012 to 2022 and generate a plot
for year in range(2012, 2023):
    # Construct the file name for the current year
    bfsfileNAME = f'https://raw.githubusercontent.com/GRUPPE-BINA24/Datenspeicher/main/Wohnbev%C3%B6lkerung_Zins{year}-neu.csv'
    # Read the data DataFrame for the current year
    bfsdf = pd.read_csv(bfsfileNAME, sep=';')
    bfsdf['KantonNr'] = bfsdf['KantonNr'].astype(int)  # Ensure the key column is integer if necessary

    # Add KantonKürzel to bfsdf
    bfsdf['KantonKürzel'] = bfsdf['KantonNr'].map(kanton_mapping)

    # Merge DataFrames on 'KantonNr'
    joined_geodf = geodf.merge(bfsdf[['KantonNr', 'KantonKürzel', 'Preis pro m2']], on='KantonNr')

    # Plotting the choropleth map with hover data
    fig = px.choropleth(
        joined_geodf,
        geojson=joined_geodf.geometry,
        locations=joined_geodf.index,
        color="Preis pro m2",
        color_continuous_scale="Greens",  # Set color scale: light green to dark green
        projection="mercator",
        # title=f"Schweizer Kantone - Preis pro m² {year}",
        hover_name="KantonKürzel",  # Show KantonKürzel when hovering
        hover_data={"KantonNr": False, "KantonKürzel": True, "Preis pro m2": True}  # Exclude KantonNr, include KantonKürzel and Preis pro m2
    )

    # Update the layout to place the title below the map and adjust the map size
    fig.update_layout(
        annotations=[dict(
            text=f"Durchschnittlicher Mietpreis pro m² (CHF) im Jahr {year}",
            x=0.5,
            y=-0.1,  # Position of the title below the map
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(
                size=20
            )
        )],
        margin={"r":120, "t":20, "l":20, "b":120},  # Adjust margins for better spacing
        height=800,  # Adjust height to make the map bigger
        width=1200,   # Adjust width to make the map bigger
        coloraxis_colorbar=dict(
            title="Preis pro m² (CHF)",  # Add the CHF unit to the color bar
            tickvals=[10, 12, 14, 16, 18, 20, 22],
            ticktext=["10 CHF", "12 CHF", "14 CHF", "16 CHF", "18 CHF", "20 CHF", "22 CHF"]
        )
    )

    fig.update_geos(fitbounds="locations")

    # Store the figure in the dictionary
    figures[year] = fig

# Display figures for all years from 2012 to 2022
for year, fig in figures.items():
    fig.show()


Output hidden; open in https://colab.research.google.com to view.